Identificar correctamente todas las diferentes variantes de fecha codificadas en este conjunto de datos, normalizar y ordenar adecuadamente las fechas. Cada línea del archivo dates.txt corresponde a una nota médica. Cada nota tiene una fecha que debe extraerse, pero cada fecha está codificada en uno de muchos formatos. 

Aquí hay una lista de algunas de las variantes que puede encontrar en este conjunto de datos:

- 04/20/2009; 04/20/09; 4/20/09; 4/3/09
- Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
- 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
- Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
- Feb 2009; Sep 2009; Oct 2010
- 6/2008; 12/2009
- 2009; 2010
 
Una vez que haya extraído estos patrones de fecha del texto, el siguiente paso es clasificarlos en orden cronológico ascendente de acuerdo con las siguientes reglas:

- Todas las fechas están en formato xx/xx/ xx son mm/dd/aa
- Todas las fechas en las que el año está codificado en solo dos dígitos corresponden a años posteriores a la década de 1900 (p. Ej., 1/5/89 es el 5 de enero de 1989).
- Si falta el día (p. Ej., 9/2009), suponga que es el primer día del mes (p. Ej., septiembre, 1 de 2009).
- Si falta el mes (por ejemplo, 2010), suponga que es el primero de enero de ese año (p. Ej., enero, 1 de 2010).
- Tenga cuidado con los posibles errores tipográficos, ya que este es un conjunto de datos derivados de la vida real.
 
Esta función debería devolver una lista de longitud 500.

In [33]:
import pandas as pd 
# importar el documento dates.txt 
doc = []
with open('dates.txt') as file:
  for line in file:
    doc.append(line)

df = pd.Series(doc)
df.head(20)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [ ]:
import re

lineas = []
fechas_extraidas = []

for linea in df:
    match1 = re.findall(r'\d{1,2}[\/\- \s]\d{1,2}[\/\- \s]\d{2,4}', linea)
    match2 = re.findall(
        r'(?:\d{1,2} )?(?:de )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:de )?(?:\d{2}, )?\d{4}',linea)
    match3 = re.findall(
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,-]*'
        r'(?:\d{1,2}(?:st|nd|rd|th)?[ ,.-]*)?'
        r'\d{4}', 
        linea
    )
    match4 = re.findall(r'\d{1,2}/\d{4}', linea)
    
    match5 = re.findall(r'\d{4}', linea)


    # Unimos todas las coincidencias y quitamos duplicados
    fechas = list(set(match1 + match2 + match3 + match4 + match5))

    lineas.append(linea.strip())
    fechas_extraidas.append(', '.join(fechas) if fechas else '')


df_fechas = pd.DataFrame({'linea': lineas, 'fecha_extraida': fechas_extraidas})
df_fechas['fecha_extraida'] = df_fechas['fecha_extraida'].str.strip()

# DataFrames separados
df_con_fechas = df_fechas[df_fechas['fecha_extraida'] != '']
df_sin_fechas = df_fechas[df_fechas['fecha_extraida'] == '']

df_con_fechas


,linea,fecha_extraida
0,03/25/93 Total time of visit (in minutes):,03/25/93
1,6/18/85 Primary Care Doctor:,6/18/85
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71
3,7 on 9/27/75 Audit C Score Current:,9/27/75
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96
...,...,...
495,1979 Family Psych History: Family History of S...,1979
496,therapist and friend died in ~2006 Parental/Ca...,2006
497,2008 partial thyroidectomy,2008
498,sPt describes a history of sexual abuse as a c...,2005


In [ ]:
meses = {
    "jan": "01", "january": "01",
    "feb": "02", "february": "02",
    "mar": "03", "march": "03",
    "apr": "04", "april": "04",
    "may": "05",
    "jun": "06", "june": "06",
    "jul": "07", "july": "07",
    "aug": "08", "august": "08",
    "sep": "09", "sept": "09", "september": "09",
    "oct": "10", "october": "10",
    "nov": "11", "november": "11",
    "dec": "12", "december": "12",
    
    # en español en caso de que asi queramos 
    "ene": "01", "enero": "01",
    "febrero": "02",
    "marzo": "03",
    "abr": "04", "abril": "04",
    "mayo": "05",
    "jun": "06", "junio": "06",
    "jul": "07", "julio": "07",
    "ago": "08", "agosto": "08",
    "sep": "09", "septiembre": "09",
    "oct": "10", "octubre": "10",
    "nov": "11", "noviembre": "11",
    "dic": "12", "diciembre": "12"
}


In [46]:
def normalizar(fecha):
    fecha = fecha.strip().lower()

    # caso dd/mm/yyyy o dd-mm-yyyy
    if re.match(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2,4}$', fecha):
        d, m, y = re.split(r'[/-]', fecha)
        if len(y) == 2:  
            y = "19" + y  
        return f"{int(d):02d}/{int(m):02d}/{y}"

    # caso con mes en texto (12 de agosto 2021, Aug 12 2021)
    match = re.match(r'(\d{1,2})\s*(?:de\s*)?([a-z]+)[a-z]*[ ,.-]*(\d{2,4})', fecha)
    if match:
        d, mes_txt, y = match.groups()
        m = meses.get(mes_txt[:3], None)
        if not m:
            return None
        if len(y) == 2:
            y = "19" + y   
        return f"{int(d):02d}/{m}/{y}"

    # caso tipo "Aug 12, 2021"
    match = re.match(r'([a-z]+)[a-z]*[ .,-]*(\d{1,2})(?:st|nd|rd|th)?[ ,.-]*(\d{2,4})', fecha)
    if match:
        mes_txt, d, y = match.groups()
        m = meses.get(mes_txt[:3], None)
        if not m:
            return None
        if len(y) == 2:
            y = "19" + y 
        return f"{int(d):02d}/{m}/{y}"

    # caso mm/yyyy → falta el día → usar 01
    match = re.match(r'^(\d{1,2})/(\d{2,4})$', fecha)
    if match:
        m, y = match.groups()
        if len(y) == 2:
            y = "19" + y   
        return f"01/{int(m):02d}/{y}"

    # caso yyyy → falta mes y día → usar 01/01/yyyy
    match = re.match(r'^\d{4}$', fecha)
    if match:
        y = match.group(0)
        return f"01/01/{y}"

    return None

In [47]:
df_con_fechas['fecha_normalizada'] = df_con_fechas['fecha_extraida'].apply(
    lambda x: ', '.join(filter(None, [normalizar(f) for f in x.split(',')]))
)

In [51]:
df_con_fechas 

,linea,fecha_extraida,fecha_normalizada
0,03/25/93 Total time of visit (in minutes):,03/25/93,03/25/1993
1,6/18/85 Primary Care Doctor:,6/18/85,06/18/1985
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,07/08/1971
3,7 on 9/27/75 Audit C Score Current:,9/27/75,09/27/1975
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96,02/06/1996
...,...,...,...
495,1979 Family Psych History: Family History of S...,1979,01/01/1979
496,therapist and friend died in ~2006 Parental/Ca...,2006,01/01/2006
497,2008 partial thyroidectomy,2008,01/01/2008
498,sPt describes a history of sexual abuse as a c...,2005,01/01/2005


In [53]:
df_ordenado = df_con_fechas['fecha_normalizada'].reset_index(drop=True)
df_ordenado

0      03/25/1993
1      06/18/1985
2      07/08/1971
3      09/27/1975
4      02/06/1996
          ...    
495    01/01/1979
496    01/01/2006
497    01/01/2008
498    01/01/2005
499    01/01/1980
Name: fecha_normalizada, Length: 500, dtype: object